In [1]:
pip install vaderSentiment

     |████████████████████████████████| 133kB 5.6MB/s 


In [2]:
#Contributors: Michel H. & Kaan A.

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [120]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Womens Clothing E-Commerce Reviews.csv to Womens Clothing E-Commerce Reviews.csv


In [61]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Womens Clothing E-Commerce Reviews.csv']))
# Dataset is now stored in a Pandas Dataframe
df
df["Title"].isnull().sum()


3810

In [62]:
df

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [63]:
df=df[['Review Text','Rating']]
df.dropna(inplace=True)


In [64]:
def encoding(x):
  if x in [1,2,3]:
    return "bad"
  else:
    return "good"

In [65]:
df['value']=df['Rating'].apply(lambda x:encoding(x))

In [66]:
df

,Review Text,Rating,value
0,Absolutely wonderful - silky and sexy and comf...,4,good
1,Love this dress! it's sooo pretty. i happene...,5,good
2,I had such high hopes for this dress and reall...,3,bad
3,"I love, love, love this jumpsuit. it's fun, fl...",5,good
4,This shirt is very flattering to all due to th...,5,good
...,...,...,...
23481,I was very happy to snag this dress at such a ...,5,good
23482,"It reminds me of maternity clothes. soft, stre...",3,bad
23483,"This fit well, but the top was very see throug...",3,bad
23484,I bought this dress for a wedding i have this ...,3,bad


In [67]:
len(df[df["Rating"]==3])

2823

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22641 entries, 0 to 23485
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  22641 non-null  object
 1   Rating       22641 non-null  int64 
 2   value        22641 non-null  object
dtypes: int64(1), object(2)
memory usage: 707.5+ KB


In [69]:
df["value"].value_counts()

good    17448
bad      5193
Name: value, dtype: int64

In [70]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stemmer = nltk.stem.porter.PorterStemmer()
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [71]:
df['words_count'] = df['Review Text'].map(lambda t: len(t.split(' '))) #split text to make list of words, and len of list is the count

In [72]:
df['punct_count'] = df['Review Text'].map(lambda t: len([x for x in t if x in (';', ':', ',', '.', '!', '?')]))

In [73]:
df['stopwords_count'] = df['Review Text'].map(lambda t: len([x for x in t.split(" ") if x in stopwords]))

In [74]:
df.iloc[]

,Review Text,Rating,value,words_count,punct_count,stopwords_count
0,Absolutely wonderful - silky and sexy and comf...,4,good,8,0,2
1,Love this dress! it's sooo pretty. i happene...,5,good,67,7,30
2,I had such high hopes for this dress and reall...,3,bad,98,9,45
3,"I love, love, love this jumpsuit. it's fun, fl...",5,good,22,8,6
4,This shirt is very flattering to all due to th...,5,good,36,5,19
...,...,...,...,...,...,...
23481,I was very happy to snag this dress at such a ...,5,good,28,2,16
23482,"It reminds me of maternity clothes. soft, stre...",3,bad,37,12,12
23483,"This fit well, but the top was very see throug...",3,bad,42,6,21
23484,I bought this dress for a wedding i have this ...,3,bad,86,9,46


In [75]:
for i in df.iloc[3]["Review Text"].split(" "):
  if i in stopwords:
    print(i)

this
it's
and
i
i
but


In [76]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

# Let's take a look at the updated text
df['Review Text']=df['Review Text'].apply(round1)


# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)


# Let's take a look at the updated text
df['Review Text']=df['Review Text'].apply(round2)



def clean_text(t):    
    tokenized_text = nltk.tokenize.word_tokenize(t.lower())
    alphabetic_text = [w for w in tokenized_text if w.isalpha()]
    alphabetic_text
    t = [w for w in alphabetic_text if w not in stopwords]
    stemmed = [stemmer.stem(w) for w in t]
    new_text = ' '.join(stemmed)
    return new_text
round3=lambda x: clean_text(x)

df['Review Text']=df['Review Text'].apply(round3)
df

,Review Text,Rating,value,words_count,punct_count,stopwords_count
0,absolut wonder silki sexi comfort,4,good,8,0,2
1,love dress sooo pretti happen find store im gl...,5,good,67,7,30
2,high hope dress realli want work initi order p...,3,bad,98,9,45
3,love love love jumpsuit fun flirti fabul everi...,5,good,22,8,6
4,shirt flatter due adjust front tie perfect len...,5,good,36,5,19
...,...,...,...,...,...,...
23481,happi snag dress great price easi slip flatter...,5,good,28,2,16
23482,remind matern cloth soft stretchi shini materi...,3,bad,37,12,12
23483,fit well top see never would work im glad abl ...,3,bad,42,6,21
23484,bought dress wed summer cute unfortun fit isnt...,3,bad,86,9,46


In [77]:
df.head()

,Review Text,Rating,value,words_count,punct_count,stopwords_count
0,absolut wonder silki sexi comfort,4,good,8,0,2
1,love dress sooo pretti happen find store im gl...,5,good,67,7,30
2,high hope dress realli want work initi order p...,3,bad,98,9,45
3,love love love jumpsuit fun flirti fabul everi...,5,good,22,8,6
4,shirt flatter due adjust front tie perfect len...,5,good,36,5,19


In [78]:
# Create quick lambda functions to find the polarity and subjectivity of each speech
from textblob import TextBlob

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

df['polarity'] = df['Review Text'].apply(pol)
df['subjectivity'] = df['Review Text'].apply(sub)
df.head()

,Review Text,Rating,value,words_count,punct_count,stopwords_count,polarity,subjectivity
0,absolut wonder silki sexi comfort,4,good,8,0,2,0.000000,0.000000
1,love dress sooo pretti happen find store im gl...,5,good,67,7,30,0.462500,0.712500
2,high hope dress realli want work initi order p...,3,bad,98,9,45,0.069412,0.388065
3,love love love jumpsuit fun flirti fabul everi...,5,good,22,8,6,0.520000,0.550000
4,shirt flatter due adjust front tie perfect len...,5,good,36,5,19,0.458333,0.658333


In [79]:
def getSIA(text):
  sia=SentimentIntensityAnalyzer()
  sentiment=sia.polarity_scores(text)
  return sentiment

In [80]:
getSIA("michel is a good person")

{'compound': 0.4404, 'neg': 0.0, 'neu': 0.58, 'pos': 0.42}

In [81]:
df=df.reset_index()

In [82]:
df

,index,Review Text,Rating,value,words_count,punct_count,stopwords_count,polarity,subjectivity
0,0,absolut wonder silki sexi comfort,4,good,8,0,2,0.000000,0.000000
1,1,love dress sooo pretti happen find store im gl...,5,good,67,7,30,0.462500,0.712500
2,2,high hope dress realli want work initi order p...,3,bad,98,9,45,0.069412,0.388065
3,3,love love love jumpsuit fun flirti fabul everi...,5,good,22,8,6,0.520000,0.550000
4,4,shirt flatter due adjust front tie perfect len...,5,good,36,5,19,0.458333,0.658333
...,...,...,...,...,...,...,...,...,...
22636,23481,happi snag dress great price easi slip flatter...,5,good,28,2,16,0.800000,0.750000
22637,23482,remind matern cloth soft stretchi shini materi...,3,bad,37,12,12,0.146429,0.660714
22638,23483,fit well top see never would work im glad abl ...,3,bad,42,6,21,0.550000,0.662500
22639,23484,bought dress wed summer cute unfortun fit isnt...,3,bad,86,9,46,0.343750,0.450000


In [83]:
df.drop(columns=["index"],inplace=True)

In [84]:
df

,Review Text,Rating,value,words_count,punct_count,stopwords_count,polarity,subjectivity
0,absolut wonder silki sexi comfort,4,good,8,0,2,0.000000,0.000000
1,love dress sooo pretti happen find store im gl...,5,good,67,7,30,0.462500,0.712500
2,high hope dress realli want work initi order p...,3,bad,98,9,45,0.069412,0.388065
3,love love love jumpsuit fun flirti fabul everi...,5,good,22,8,6,0.520000,0.550000
4,shirt flatter due adjust front tie perfect len...,5,good,36,5,19,0.458333,0.658333
...,...,...,...,...,...,...,...,...
22636,happi snag dress great price easi slip flatter...,5,good,28,2,16,0.800000,0.750000
22637,remind matern cloth soft stretchi shini materi...,3,bad,37,12,12,0.146429,0.660714
22638,fit well top see never would work im glad abl ...,3,bad,42,6,21,0.550000,0.662500
22639,bought dress wed summer cute unfortun fit isnt...,3,bad,86,9,46,0.343750,0.450000


In [85]:


compound=[]
neg=[]
pos=[]
neu=[]
SIA=0
for i in range(0,len(df["Review Text"])):
  SIA = getSIA(df["Review Text"][i])
  compound.append(SIA['compound'])
  neg.append(SIA['neg'])
  neu.append(SIA['neu'])
  pos.append(SIA['pos'])

In [86]:
df["compound"]=compound
df["neg"]=neg
df["neu"]=neu
df["pos"]=pos


In [129]:
df['neg'].sort_values(ascending=False)

10819    0.655
4933     0.574
5389     0.565
14222    0.545
14264    0.531
         ...  
13048    0.000
13047    0.000
13046    0.000
13044    0.000
0        0.000
Name: neg, Length: 22641, dtype: float64

In [130]:
df.iloc[10819]

Review Text        low waist weird liner
Rating                                 3
value                                bad
words_count                            4
punct_count                            2
stopwords_count                        0
polarity                           -0.25
subjectivity                        0.65
compound                         -0.4215
neg                                0.655
neu                                0.345
pos                                    0
Name: 10819, dtype: object

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=2000)
X=tfidf.fit_transform(df['Review Text'])

In [89]:
new_features = df[['words_count','punct_count','stopwords_count','polarity'	,'subjectivity',	'compound',	'neg'	,'neu',	'pos']].to_numpy()
new_features

array([[ 8.   ,  0.   ,  2.   , ...,  0.   ,  0.615,  0.385],
       [67.   ,  7.   , 30.   , ...,  0.   ,  0.647,  0.353],
       [98.   ,  9.   , 45.   , ...,  0.   ,  0.719,  0.281],
       ...,
       [42.   ,  6.   , 21.   , ...,  0.   ,  0.509,  0.491],
       [86.   ,  9.   , 46.   , ...,  0.108,  0.655,  0.237],
       [19.   ,  4.   ,  6.   , ...,  0.   ,  0.465,  0.535]])

In [90]:
new_x = np.concatenate((X.toarray(),new_features), axis=1)

In [91]:
new_x.shape

(22641, 2009)

In [92]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(df[['value']])
y

array([1, 1, 0, ..., 0, 0, 1])

In [93]:
 from sklearn import preprocessing
scaler=preprocessing.StandardScaler().fit(new_x)
new_x=scaler.transform(new_x)



In [94]:
from imblearn.over_sampling import SMOTE

# Resample the minority class. You can change the strategy to 'auto' if you are not sure.
sm = SMOTE(sampling_strategy='minority', random_state=7)

# Fit the model to generate the data.
oversampled_trainX, oversampled_trainY = sm.fit_sample(new_x, y)


In [95]:
count1=0
count2=0
for i in oversampled_trainY:
  if i ==1:
    count1+=1
  else:
    count2+=1
print(count1)
print(count2)



17448
17448


In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(oversampled_trainX,oversampled_trainY,train_size=0.8)

In [97]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((27916, 2009), (6980, 2009), (27916,), (6980,))

In [98]:
X_train[0].dtype

dtype('float64')

In [99]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

In [100]:
model = Sequential()
model.add(Input(shape=2009))
model.add(Dense(1000))
model.add(Dense(100))
model.add(Dense(1, activation='sigmoid'))

In [101]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1000)              2010000   
_________________________________________________________________
dense_4 (Dense)              (None, 100)               100100    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 2,110,201
Trainable params: 2,110,201
Non-trainable params: 0
_________________________________________________________________


In [102]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'])

In [103]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test),batch_size=256)

Epoch 1/20
110/110 [==============================] - 9s 77ms/step - loss: 0.6571 - accuracy: 0.7673 - val_loss: 0.4462 - val_accuracy: 0.8542
Epoch 2/20
110/110 [==============================] - 10s 91ms/step - loss: 0.3623 - accuracy: 0.8603 - val_loss: 0.4055 - val_accuracy: 0.8572
Epoch 3/20
110/110 [==============================] - 8s 72ms/step - loss: 0.3166 - accuracy: 0.8800 - val_loss: 0.3519 - val_accuracy: 0.8629
Epoch 4/20
110/110 [==============================] - 7s 67ms/step - loss: 0.2975 - accuracy: 0.8853 - val_loss: 0.4973 - val_accuracy: 0.7576
Epoch 5/20
110/110 [==============================] - 7s 67ms/step - loss: 0.2938 - accuracy: 0.8846 - val_loss: 0.3463 - val_accuracy: 0.8668
Epoch 6/20
110/110 [==============================] - 7s 67ms/step - loss: 0.2698 - accuracy: 0.8968 - val_loss: 0.3420 - val_accuracy: 0.8564
Epoch 7/20
110/110 [==============================] - 7s 68ms/step - loss: 0.2661 - accuracy: 0.8992 - val_loss: 0.3311 - val_accuracy: 0.878

In [104]:
                                          

stemmer = nltk.stem.porter.PorterStemmer()
stopwords = nltk.corpus.stopwords.words('english')
def clean_text(text): 
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers and remove end of lines.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    tokenized_text = nltk.tokenize.word_tokenize(text.lower())
    alphabetic_text = [w for w in tokenized_text if w.isalpha()]
    alphabetic_text
    text = [w for w in alphabetic_text if w not in stopwords]
    stemmed = [stemmer.stem(w) for w in text]
    new_text = ' '.join(stemmed)
    return new_text

In [105]:
TextBlob("hello").sentiment.subjectivity

0.0

In [106]:
model.evaluate(X_test,y_test)[1]

219/219 [==============================] - 1s 6ms/step - loss: 0.3178 - accuracy: 0.8821


0.8820917010307312

In [125]:
p=[]
for i in model.predict(X_test):
  if i <0.5:
    p.append(0)
  else:
    p.append(1)

confusion_matrix(y_test,p)


array([[3123,  297],
       [ 526, 3034]])

In [117]:
def predict_text():
    text = input("Enter review: ")
    text_features = tfidf.transform([clean_text(text)])
    words_count = len(text.split(' '))
    punct_count = len([x for x in text if x in (';', ':', ',', '.', '!', '?')])
    stopwords_count = len([x for x in text.split(" ") if x in stopwords])
    SIA=getSIA(text)
    pol = TextBlob(text).sentiment.polarity
    sub = TextBlob(text).sentiment.subjectivity
    compound=SIA['compound']
    neg=SIA['neg']
    neu=SIA['neu']
    pos=SIA['pos']
    new=pd.DataFrame({'words_count':words_count,'punct_count':punct_count,'stopwords_count':stopwords_count,'polarity':pol	,'subjectivity':sub,	'compound':compound,	'neg':neg	,'neu':neu,	'pos':pos}, index=[0])
    new_features1 = new[['words_count','punct_count','stopwords_count','polarity'	,'subjectivity',	'compound',	'neg'	,'neu',	'pos']].to_numpy()
    new_features2 = np.concatenate((text_features.toarray(),new_features1), axis=1)
    new_features2=scaler.transform(new_features2)
    prediction = model.predict(new_features2)
    prediction_rounded= round(prediction[0][0])
    if prediction <=0.4 :
      return "with a {:.2f}% confidence the review is {}".format((1-prediction[0][0])*100,encoder.inverse_transform([prediction_rounded])[0])
    if prediction >=0.6 :
      return "with a {:.2f}% confidence the review is {}".format((prediction[0][0])*100,encoder.inverse_transform([prediction_rounded])[0])
    else:
      return f"can't significantly predict if it's good or bad {(prediction[0][0])*100}"


In [108]:
df.iloc[22639]["Review Text"]

'bought dress wed summer cute unfortun fit isnt perfect medium fit waist perfectli way long big bust shoulder want spend money could get tailor felt like might worth side note dress deliv nordstrom tag found much cheaper look'

In [109]:
round(0.3)

0

In [126]:
predict_text()

Enter review: i am angry about this company and it's bad


'with a 90.44% confidence the review is bad'

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.ensemble import AdaBoostClassifier


In [ ]:
# model_MLP=MLPClassifier(activation='logistic',solver='adam',max_iter=100)
# model_MLP.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=100,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
# from sklearn.metrics import accuracy_score
# test_predictions=model_MLP.predict(X_test)
# test_accuracy = accuracy_score(test_predictions,y_test)
# print ("validation Accuracy : {0:.4%}".format(test_accuracy))

validation Accuracy : 85.8688%


In [ ]:
# clf = GridSearchCV(model_MLP, {
#             'activation' : ['identity', 'logistic', 'tanh', 'relu'],
#             'solver' : ['lbfgs', 'sgd', 'adam']
#                                }, cv=5, return_train_score=False)
# clf.fit(X_test, y_test)
# dfcv = pd.DataFrame(clf.cv_results_)[['param_activation','param_solver','mean_test_score']].sort_values('mean_test_score')
# display(dfcv)
# print(f"best params: {clf.best_params_}")
# print(f"best score: {clf.best_score_}")


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimiza

,param_activation,param_solver,mean_test_score
9,relu,lbfgs,0.324945
0,identity,lbfgs,0.506182
10,relu,sgd,0.635710
1,identity,sgd,0.684275
6,tanh,lbfgs,0.749615
3,logistic,lbfgs,0.756684
4,logistic,sgd,0.769927
7,tanh,sgd,0.769927
8,tanh,adam,0.818059
2,identity,adam,0.820487


best params: {'activation': 'logistic', 'solver': 'adam'}
best score: 0.8374900296366763


In [ ]:
# def predict_text_MLP():
#     text = input("Enter review: ")
#     text_features = tfidf.transform([clean_text(text)])
#     words_count = len(text.split(' '))
#     punct_count = len([x for x in text if x in (';', ':', ',', '.', '!', '?')])
#     stopwords_count = len([x for x in text if x in stopwords])
#     SIA=getSIA(text)
#     pol = TextBlob(text).sentiment.polarity
#     sub = TextBlob(text).sentiment.subjectivity
#     compound=SIA['compound']
#     neg=SIA['neg']
#     neu=SIA['neu']
#     pos=SIA['pos']
#     new=pd.DataFrame({'words_count':words_count,'punct_count':punct_count,'stopwords_count':stopwords_count,'polarity':pol	,'subjectivity':sub,	'compound':compound,	'neg':neg	,'neu':neu,	'pos':pos}, index=[0])
#     new_features1 = new[['words_count','punct_count','stopwords_count','polarity'	,'subjectivity',	'compound',	'neg'	,'neu',	'pos']].to_numpy()
#     new_features2 = np.concatenate((text_features.toarray(),new_features1), axis=1)
#     prediction = model_MLP.predict(new_features2)
#     return encoder.inverse_transform(prediction)[0][0]
    


In [ ]:
# predict_text_MLP()

Enter review: hihi


'good'

In [ ]:


# model = RandomForestClassifier(criterion="entropy")
# model.fit(X_train, y_train)



RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# from sklearn.metrics import accuracy_score
# test_predictions=model.predict(X_test)
# test_accuracy = accuracy_score(test_predictions,y_test)
# print ("validation Accuracy : {0:.4%}".format(test_accuracy))

validation Accuracy : 82.9543%


In [ ]:
# from keras.preprocessing.sequence import pad_sequences

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# # Keras
# from keras.layers import Dense
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation
# from keras.layers import Embedding, Flatten
# from keras.layers import Conv1D, GlobalMaxPooling1D
# from keras.layers import LSTM
# from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
# from keras.preprocessing.sequence import pad_sequences
# from xgboost import XGBClassifier

In [ ]:
# def build_model(vocab_size, dim, embedding_matrix, max_length):
#     model = Sequential()
#     model.add(Embedding(vocab_size + 1, dim, weights=[embedding_matrix], input_length=max_length))
#     model.add(Dropout(0.4))
#     model.add(LSTM(128))
#     model.add(Dense(64))
#     model.add(Dropout(0.5))
#     model.add(Activation('relu'))
#     model.add(Dense(2))
#     model.add(Activation('sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model



In [ ]:
# model = build_model(vocab_size, dim, embedding_matrix, max_length)

In [ ]:
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)
# model.fit(X_train, y_train, batch_size=32, epochs=8, validation_split=0.1, shuffle=True, callbacks=[reduce_lr])

In [ ]:
neg_words=["sad","anxious","down", "stressed" , "upset","pain","crazy","fault","can't","cannot","loneliness","sadness","rage"]
b=input("write aBOUT YOUR SELF:")
b=b.split()
w=[]
binary=0
for i in b:
    if i.lower() in neg_words:
        w.append(i)
        binary=1
print(w)
print (binary)


write aBOUT YOUR SELF:i am sad and stressed
['sad', 'stressed']
1
